<a href="https://colab.research.google.com/github/SanghunOh/study_machinglearning/blob/main/codes/sanghunoh/reports/%EA%B3%BC%EC%A0%9C_2_2_iris_KNN%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 2. (50점) 각 클래스당 데이터 50개씩 총 3개의 클래스로 이루어진 iris 데이터를 LMS 과제 게시판에서 다운로드 받고, K-근접이웃 분류기를 이용하여 iris 데이터를 분류하시오. 데이터의 거리 계산은 유클리디안 거리(euclidean distance)를 사용할 것, K값을 1, 5, 10, 15, 20, 25, 30으로 변화시키면서 수행하고, 그 결과를 표로 정리하여 비교하시오

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# load data
refer iris : https://archive.ics.uci.edu/ml/datasets/iris

In [2]:
# 작업 경로 설정
import os
os.chdir(r"/content/drive/MyDrive/Colab Notebooks/00.study/10.knou")
!pwd
# !ls

/content/drive/MyDrive/Colab Notebooks/00.study/10.knou


In [3]:
!unzip -o ./datas/iris.zip -d ./datas
!ls ./datas/*.mat

Archive:  ./datas/iris.zip
  inflating: ./datas/iris_shuffle.mat  
  inflating: ./datas/matlab_iris_shuffle.mat  
./datas/dataCh4_7.mat	  ./datas/matlab_iris_shuffle.mat
./datas/iris_shuffle.mat


## EDA (Exploratory Data Analysis) 탐색적 데이터 분석

In [4]:
# read dataset file
import scipy.io
mat = scipy.io.loadmat('./datas/matlab_iris_shuffle.mat')
# mat = scipy.io.loadmat('./datas/iris_shuffle.mat')

type(mat), mat.keys(), 

(dict,
 dict_keys(['__header__', '__version__', '__globals__', 'iris_class', 'iris_data']))

In [5]:
import numpy as np

In [6]:
for key in list(mat.keys()):
  result = None
  if isinstance(mat[key], np.ndarray):
    result = mat[key].shape
  else :
    result = mat[key]
  print('key name:{}, result:{}, data type:{}'.format(key ,result, type(mat[key])))

key name:__header__, result:b'MATLAB 5.0 MAT-file, written by Octave 5.2.0, 2020-03-26 12:54:05 UTC', data type:<class 'bytes'>
key name:__version__, result:1.0, data type:<class 'str'>
key name:__globals__, result:[], data type:<class 'list'>
key name:iris_class, result:(150, 1), data type:<class 'numpy.ndarray'>
key name:iris_data, result:(150, 4), data type:<class 'numpy.ndarray'>


In [7]:
class_type, class_count = np.unique(mat['iris_class'],return_counts=True)
class_type, class_count

(array([array([[1.]]), array([[2.]]), array([[3.]])], dtype=object),
 array([50, 50, 50]))

In [8]:
import pandas as pd

In [9]:
iris_pd = pd.DataFrame(mat['iris_data'])
iris_pd.info()
# iris_pd.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       150 non-null    object
 1   1       150 non-null    object
 2   2       150 non-null    object
 3   3       150 non-null    object
dtypes: object(4)
memory usage: 4.8+ KB


### 추론 : 3개 Class이며, 각 데이터 row는 50개, 입력변수 갯수는 4개, mat['iris_data']와 mat['iris_class']는 각 row가 매칭

In [10]:
# dtype 일치 위해 변환
iris_data = mat['iris_data'].astype('float64')
iris_class = mat['iris_class'].astype('int32')

In [11]:
# iris = np.hstack([mat['iris_data'], mat['iris_class']])
iris = np.concatenate([iris_data, iris_class],axis=1)
iris.shape

(150, 5)

## K-근접이웃 분류기

### kernel=5 로 실험

In [12]:
iris[3,0:5], np.linalg.norm(iris[2:4,0:4] - iris[3:5,0:4], axis=1)

(array([5.9, 3.2, 4.8, 1.8, 2. ]), array([1.19582607, 2.71477439]))

In [13]:
from pandas._libs.algos import ensure_uint8
kernel_count = 20
error_count = np.full((3),0,dtype=np.int64) # class A,B,C
for item in iris:
  # 유클리디안 거리(euclidean distance) 사용해 모든 값과 거리 계산
  distance_norm = np.linalg.norm(iris[:,0:4] - item[0:4], axis=1)
  # 거리 계산과 결과 변수(class type)를 merge
  kernel_stack = np.stack((distance_norm, iris[:,4]), axis=1)
  kernel_items = kernel_stack[kernel_stack[:,0].argsort()]  # 거리 계산 column 기준 가까운 거리로 정렬
  class_types = np.full((3),0)
  for kernel_item in kernel_items[1:kernel_count+1,:]:  # 비교 item 거리는 제외 위해 [0] 제외
    class_type = int(kernel_item[1] - 1) # class_types 에 담기 위해 -1
    # print(class_type,kernel_item[1])
    class_types[class_type] = class_types[class_type] + 1
  # 오류 데이터 측정
  max_idx = np.argmax(class_types) + 1
  if item[4] != max_idx:
    item_target = int(item[1]-1)
    # print(item[4], item_target)
    error_count[item_target] = error_count[item_target] + 1
  pass

kernel_stack[0:10,:]
# distance_norm.shape, item[0:4]
# kernel_items[0:10,:]
# print('error rate : {}'.format(error_count/class_count[0]))
# print('error rate : {}'.format(np.mean(error_count)))

array([[1.56843871, 2.        ],
       [4.90509939, 1.        ],
       [0.34641016, 3.        ],
       [1.28452326, 2.        ],
       [1.65529454, 3.        ],
       [2.54165301, 2.        ],
       [4.62817459, 1.        ],
       [2.41246762, 2.        ],
       [2.24944438, 2.        ],
       [4.55411901, 1.        ]])

In [14]:
kernel_items[0:10,:]

array([[0.        , 3.        ],
       [0.24494897, 3.        ],
       [0.26457513, 3.        ],
       [0.34641016, 3.        ],
       [0.34641016, 3.        ],
       [0.36055513, 3.        ],
       [0.37416574, 3.        ],
       [0.41231056, 3.        ],
       [0.42426407, 3.        ],
       [0.4472136 , 3.        ]])

In [15]:
np.sum(error_count) / class_count.shape[0]

1.0

### 여러 Kernel 변화 적용해 비교

In [16]:
def knn_classtify(kernel_count):
  _kernel_count = kernel_count
  error_count = np.full((3),0,dtype=np.int64) # class A,B,C
  for item in iris:
    # 유클리디안 거리(euclidean distance) 사용해 모든 값과 거리 계산
    distance_norm = np.linalg.norm(iris[:,0:4] - item[0:4], axis=1)
    # 거리 계산과 결과 변수(class type)를 merge
    kernel_stack = np.stack((distance_norm, iris[:,4]), axis=1)
    kernel_items = kernel_stack[kernel_stack[:,0].argsort()]  # 거리 계산 column 기준 가까운 거리로 정렬
    class_types = np.full((3),0)
    for kernel_item in kernel_items[1:_kernel_count+1,:]:  # 비교 item 거리는 제외 위해 [0] 제외
      class_type = int(kernel_item[1] - 1) # class_types 에 담기 위해 -1
      class_types[class_type] = class_types[class_type] + 1
    # 오류 데이터 측정
    max_idx = np.argmax(class_types) + 1
    if item[4] != max_idx:
      item_target = int(item[1]-1)
      error_count[item_target] = error_count[item_target] + 1
  return np.mean(error_count)

####K값을 1, 5, 10, 15, 20, 25, 30으로 변화

In [17]:
loop_list = [1, 5, 10, 15, 20, 25, 30]
for knl in loop_list:
  print('kernel:{}, knn_classtify:{}'.format(knl,knn_classtify(knl)))

kernel:1, knn_classtify:2.0
kernel:5, knn_classtify:1.6666666666666667
kernel:10, knn_classtify:1.6666666666666667
kernel:15, knn_classtify:1.3333333333333333
kernel:20, knn_classtify:1.0
kernel:25, knn_classtify:1.6666666666666667
kernel:30, knn_classtify:2.6666666666666665


#### 결과 비교 : Train dataset 으로 확인 시 kernel count를 20을 선택하는 것이 좋음.
|kernel count|1|5|10|15|20|25|30|
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|측정치|2.0|1.67|1.67|1.34|1.0|1.67|2.67|